In [2]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.preprocessing import LabelEncoder

df = pd.read_excel('DATA.xlsx', engine='openpyxl').drop(columns=['t25_Revenu','s118_TauxInteret','s124_DateOctroiCredit','ID CLIENT','N_DOSS','s110_Mensualite','SECTEUR','ZONE GEOGRAPHIQUE','m3_Commune'])

df.info() 

# print(df.head())

numerical_columns= ['MT_ACCORD', 'Revenu_estime','DUR_P','Age']  
categoical_columns=['m2_Wilaya','CATEGORIE','t24_Profession','CODE','t18_Genre','t23_EtatCivil','I_CLASS']



# Convertir les colonnes numériques en type numérique
for col in numerical_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
for col in categoical_columns:
    df[col] = df[col].astype(object)
    

# Afficher le nombre de valeurs manquantes pour chaque variable dans le dataset initial
print("Number of missing values per column before Imputation :\n", df.isnull().sum())

# Selecting a sample of the dataset
df = df.sample(n=100, random_state=1)


# Print the numer of missing values in the sample
print("Number of missing values in the sample before Imputation:\n", df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112605 entries, 0 to 112604
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   CODE            112605 non-null  int64  
 1   m2_Wilaya       112605 non-null  int64  
 2   MT_ACCORD       112605 non-null  float64
 3   DUR_P           112605 non-null  int64  
 4   CATEGORIE       112605 non-null  object 
 5   Age             112605 non-null  int64  
 6   t18_Genre       112605 non-null  int64  
 7   t23_EtatCivil   112259 non-null  float64
 8   t24_Profession  109701 non-null  float64
 9   I_CLASS         112605 non-null  object 
 10  Revenu_estime   112605 non-null  float64
dtypes: float64(4), int64(5), object(2)
memory usage: 9.5+ MB
Nombre de valeurs manquantes par variable avant l'imputation :
 CODE                 0
m2_Wilaya            0
MT_ACCORD            0
DUR_P                0
CATEGORIE            0
Age                  0
t18_Genre            0
t23_

In [3]:
# Separating numerical and categorical data
df_numeriques = df.select_dtypes(include=['number'])
df_categorielles = df.select_dtypes(exclude=['number'])

print(df_numeriques)
print(df_categorielles)

         MT_ACCORD  DUR_P  Age  Revenu_estime
0       10800000.0    252   55  118830.791089
1       13000000.0     60   72  642925.787124
2        8526000.0    177   48  125185.000000
3       10800000.0    228   56  155504.101673
4        8764000.0    192   59  123534.530531
...            ...    ...  ...            ...
112600   2990000.0    300   28   41548.477728
112601   2900000.0    420   40   41338.753837
112602  13500000.0    300   49  160046.318427
112603   1584000.0    300   19   26738.203589
112604   3500000.0    480   26   39336.748268

[112605 rows x 4 columns]
       CODE m2_Wilaya                          CATEGORIE t18_Genre  \
0       101        16  CREDITS HYPOTHECAIRES BONIFIES 1%         2   
1       101        16                  PRETS  EPARGNANTS         1   
2       101        16              PRETS NON  EPARGNANTS         1   
3       101        16  CREDITS HYPOTHECAIRES BONIFIES 3%         2   
4       101        16  CREDITS HYPOTHECAIRES BONIFIES 1%         1   
.

In [4]:
# Encoding categorical variables with ordinal encoder
colonnes_a_encoder = ['CATEGORIE','t24_Profession','I_CLASS']  #Selecting which columns to encode

# Ensure all data in categorical columns are of type string
for col in colonnes_a_encoder:
     df[col] = df[col].astype(str)


encoder = OrdinalEncoder()

df[colonnes_a_encoder] = encoder.fit_transform(df[colonnes_a_encoder])

#Print the encoded categories
for i, col in enumerate(colonnes_a_encoder):
    print(f"Encoding for {col}:")
    for categorie, encodage in enumerate(encoder.categories_[i]):
        print(f"{categorie} : {encodage}")
    print("\n")


# Check the encoded dataframe
print(df)
print(df.columns)

Encodage pour CATEGORIE:
0 : CREDIT LOCATION VENTE
1 : CREDITS HYPOTHECAIRES BONIFIES 1%
2 : CREDITS HYPOTHECAIRES BONIFIES 3%
3 : FINANCEMENT IJARA MOUNTAHIA BITAMLIK
4 : PRETS  EPARGNANTS
5 : PRETS NON  EPARGNANTS


Encodage pour t24_Profession:
0 : 1.0
1 : 2.0
2 : 111.0
3 : 112.0
4 : 113.0
5 : 121.0
6 : 122.0
7 : 131.0
8 : 214.0
9 : 221.0
10 : 222.0
11 : 231.0
12 : 232.0
13 : 233.0
14 : 241.0
15 : 242.0
16 : 243.0
17 : 244.0
18 : 245.0
19 : 311.0
20 : 312.0
21 : 313.0
22 : 316.0
23 : 321.0
24 : 323.0
25 : 324.0
26 : 331.0
27 : 334.0
28 : 341.0
29 : 342.0
30 : 343.0
31 : 344.0
32 : 345.0
33 : 347.0
34 : 349.0
35 : 411.0
36 : 412.0
37 : 413.0
38 : 414.0
39 : 419.0
40 : 421.0
41 : 422.0
42 : 511.0
43 : 512.0
44 : 514.0
45 : 516.0
46 : 517.0
47 : 522.0
48 : 611.0
49 : 612.0
50 : 613.0
51 : 615.0
52 : 621.0
53 : 712.0
54 : 713.0
55 : 733.0
56 : 742.0
57 : 821.0
58 : 829.0
59 : 832.0
60 : 833.0
61 : 921.0
62 : nan


Encodage pour I_CLASS:
0 : C 0
1 : C 1
2 : C 2
3 : C 3


       CODE m2_W

In [5]:

df['t24_Profession'].replace(62, np.nan, inplace=True)


# Using Iterative Imputer on all columns for imputation of missing values
imputer = IterativeImputer(estimator=RandomForestRegressor(), max_iter=10, random_state=0)
df_imputed = imputer.fit_transform(df)

# Convert result to a pd dataframe
df_imputed = pd.DataFrame(df_imputed, columns=df.columns)

# Rounding the values in categorical columns
df_imputed[colonnes_categorielles] = df_imputed[colonnes_categorielles].round()


# Checking imputation results
print("Number of missing values after Imputation :\n", df_imputed.isnull().sum())

# Final dataframe
print(df_imputed)

Nombre de valeurs manquantes par variable après l'imputation :
 CODE              0
m2_Wilaya         0
MT_ACCORD         0
DUR_P             0
CATEGORIE         0
Age               0
t18_Genre         0
t23_EtatCivil     0
t24_Profession    0
I_CLASS           0
Revenu_estime     0
dtype: int64
         CODE  m2_Wilaya   MT_ACCORD  DUR_P  CATEGORIE   Age  t18_Genre  \
0       101.0       16.0  10800000.0  252.0        1.0  55.0        2.0   
1       101.0       16.0  13000000.0   60.0        4.0  72.0        1.0   
2       101.0       16.0   8526000.0  177.0        5.0  48.0        1.0   
3       101.0       16.0  10800000.0  228.0        2.0  56.0        2.0   
4       101.0       16.0   8764000.0  192.0        1.0  59.0        1.0   
...       ...        ...         ...    ...        ...   ...        ...   
112600  504.0        2.0   2990000.0  300.0        4.0  28.0        1.0   
112601  271.0       25.0   2900000.0  420.0        5.0  40.0        1.0   
112602  401.0       31.0  13

In [6]:
df_imputed.to_excel('DATAImputed.xlsx', index=False, engine='openpyxl')